In [0]:
import cv2
import math
import matplotlib.pyplot as plt
import pandas as pd
from keras.preprocessing import image
import numpy as np
from keras.utils import np_utils
from skimage.transform import resize

In [0]:
import re
import h5py
import json
import pickle
import random
from PIL import Image
from pickle import load

import matplotlib.pyplot as plt

import matplotlib.image as mpimg
from IPython.display import display
from keras.backend import clear_session
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.text import Tokenizer
from nltk.translate.bleu_score import corpus_bleu
from keras.utils import to_categorical, plot_model
from keras.preprocessing.sequence import pad_sequences
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.preprocessing.image import load_img, img_to_array
from keras.layers import Input, Dense, LSTM, Embedding, Dropout, add
from keras.models import Model, Sequential, model_from_json, load_model

In [0]:
import os
path="/content/video/"

In [5]:
list1=[]
f=open("e39.txt","r")
content=f.read()
for i in content.split("\n"):
  list1.append(i)

df=pd.DataFrame()
s=pd.Series(list1)    
df_namesplit=s.str.split(":",n=1,expand=True)
df_namesplit.head()

,0,1
0,maya,gud moning maanusha teevee dekhane ke lie taiy...
1,maya,anumaan hai jise dekhakar lage ki ye shaayad r...
2,sahil,sahit vhait peteekot nahin hai lekin teevee ka...
3,maya,itana sun loo ke aaj shaam ko hamaaree paartee...
4,indu,donon ka rang safed kanphyoojan hoga


In [7]:
'df=df_namesplit.drop(1,axis=1)'
df=df_namesplit
df.rename(columns={0:"Label",1:"Text"},inplace=True)
df.head()

,Label,Text
0,maya,gud moning maanusha teevee dekhane ke lie taiy...
1,maya,anumaan hai jise dekhakar lage ki ye shaayad r...
2,sahil,sahit vhait peteekot nahin hai lekin teevee ka...
3,maya,itana sun loo ke aaj shaam ko hamaaree paartee...
4,indu,donon ka rang safed kanphyoojan hoga


In [0]:
count=0
path="/content/video/"
list_label=[]
j=0
for i in os.listdir(path):
  videoFile=path+i
  cap=cv2.VideoCapture(videoFile)
  frameRate=cap.get(5)
  x=1
  os.chdir('/content/frames')
  while(cap.isOpened()):
    frameId=cap.get(1)
    ret,frame=cap.read()
    if(ret!=True):
      break  
    if(frameId%math.floor(frameRate)==0):
      filename ="{0}f{1}.jpg".format(df['Label'][j],count)
      count+=1
      cv2.imwrite(filename, frame)
      list_label.append(df['Label'][j])
  cap.release()
  j+=1  






In [9]:
name=[]
count=0
for i in os.listdir():
  name.append(i)  
  count+=1

len(name)

1407

In [10]:
data=pd.DataFrame()
data['Class']=list_label
name.sort()
data['Image_ID']=name
data=data[["Image_ID","Class"]]
data.head()

,Image_ID,Class
0,induf1000.jpg,maya
1,induf1001.jpg,maya
2,induf1002.jpg,maya
3,induf1003.jpg,maya
4,induf1004.jpg,maya


In [0]:
X=[]
for img_name in data.Image_ID:
  img=plt.imread(''+img_name)
  X.append(img)
X=np.array(X) 

In [19]:
X.shape

(1407, 299, 299, 3)

In [0]:
from keras.applications.inception_v3 import InceptionV3,preprocess_input
from keras.layers import Dense,InputLayer,Dropout

In [0]:
from keras.applications.inception_v3 import preprocess_input
X=preprocess_input(X)

In [15]:
inception_model = InceptionV3(weights='imagenet')
    #removing the top layer to get the vector representation if the image
inception_model.layers.pop()
inception_model = Model(inputs=inception_model.inputs, outputs=inception_model.layers[-1].output)

96116736/96112376 [==============================] - 2s 0us/step


In [0]:
image=[]
for i in range(0,X.shape[0]):
  a=resize(X[i],preserve_range=True,output_shape=(299,299)).astype(int)
  image.append(a)
X=np.array(image) 

In [20]:
inception_model.predict(X)

array([[0.11824495, 0.3625783 , 0.00541302, ..., 1.0514245 , 0.42888233,
        1.3778363 ],
       [0.1208288 , 0.40932027, 0.00935532, ..., 0.87156874, 0.8008306 ,
        1.1363269 ],
       [0.07730815, 0.39237592, 0.        , ..., 0.96628857, 0.44011998,
        0.8239568 ],
       ...,
       [0.01630958, 0.28816444, 0.08278149, ..., 0.17489472, 0.31159732,
        0.62627757],
       [0.02193597, 0.3628222 , 0.23739842, ..., 0.2757547 , 0.4594185 ,
        0.8132234 ],
       [0.09482712, 0.103049  , 0.13211407, ..., 0.06744358, 0.4441544 ,
        1.1656046 ]], dtype=float32)

In [34]:
os.listdir('/content/character')

['sahilf274.jpg',
 'indu.jpg',
 'indu (2).jpg',
 'mayaf1222.jpg',
 'sahil2.jpg',
 'rosheshf1074.jpg',
 'monisha.jpg',
 'sid.jpg',
 'madhu.jpg',
 'sahil.jpg',
 '.ipynb_checkpoints',
 'sahil1.jpg',
 'monisha (2).jpg',
 'mayaf1212.jpg']

In [0]:
X=[]

In [44]:
img_test=["mayaf1212.jpg","mayaf1222.jpg","indu.jpg","indu2.jpg","madhu.jpg","monisha.jpg","monisha2.jpg","sahil1.jpg","sahil2.jpg","sid.jpg","roshesh.jpg"]
os.chdir('/content/character')
X=[]
for img_name in img_test:
  img=plt.imread(''+img_name)
  X.append(img)
X=np.array(X)
image=[]
for i in range(0,X.shape[0]):
  a=resize(X[i],preserve_range=True,output_shape=(299,299)).astype(int)
  image.append(a)
X=np.array(image)
X.shape

(11, 299, 299, 3)

In [0]:
emb=inception_model.predict(X)
emb

In [50]:
m_emb=np.mean(np.array([emb[0],emb[1]]),axis=0)
m_emb.shape

(2048,)

In [51]:
i_emb=np.mean(np.array([emb[2],emb[3]]),axis=0)
i_emb.shape

(2048,)

In [53]:
madhu_emb=emb[4]
madhu_emb.shape

(2048,)

In [54]:
monisha_emb=np.mean(np.array([emb[5],emb[6]]),axis=0)
monisha_emb.shape

(2048,)

In [55]:
sahil_emb=np.mean(np.array([emb[7],emb[8]]),axis=0)
sahil_emb.shape

(2048,)

In [56]:
roshesh_emb=emb[10]
roshesh_emb.shape

(2048,)

In [57]:
sid=emb[9]
sid.shape

(2048,)

In [0]:
vis_emb={}
vis_emb["maya"]=m_emb
vis_emb["indu"]=i_emb
vis_emb["sahil"]=sahil_emb
vis_emb["monisha"]=monisha_emb
vis_emb["madhu"]=madhu_emb
vis_emb["sid"]=sid
vis_emb["roshesh"]=roshesh_emb

In [63]:
vis_emb["sahil"].shape

(2048,)

In [0]:
f=open('embedding.pkl','wb')
import pickle
pickle.dump(vis_emb,f)

In [67]:
c=open('embedding.pkl','rb')
d=pickle.load(c)
d

{'indu': array([3.5523229 , 1.3372188 , 1.5370908 , ..., 0.62386394, 0.        ,
        6.3291154 ], dtype=float32),
 'madhu': array([3.3542182 , 1.929963  , 0.41037172, ..., 0.        , 0.13121375,
        5.168471  ], dtype=float32),
 'maya': array([ 7.870367  ,  3.025898  ,  0.03965998, ...,  3.1330118 ,
         0.04394486, 17.079487  ], dtype=float32),
 'monisha': array([ 1.2650446 ,  6.365317  ,  0.6026185 , ...,  0.98808885,
         0.        , 12.793686  ], dtype=float32),
 'roshesh': array([0.8765143, 0.9946318, 3.6631622, ..., 0.       , 0.       ,
        5.126812 ], dtype=float32),
 'sahil': array([6.747183  , 6.2006083 , 0.21887946, ..., 0.04634268, 0.        ,
        6.931761  ], dtype=float32),
 'sid': array([5.5270066 , 3.9554007 , 0.80949426, ..., 0.        , 0.        ,
        2.9833195 ], dtype=float32)}